In [ ]:
import requests
from urllib.parse import urlparse
import pandas as pd
from haversine import haversine
import folium
from pandas import DataFrame,Series
import re

In [ ]:
#주소입력 -> 위도 경도 출력 함수
def getLatLng(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?&query=' + addr
    result = requests.get(urlparse(url).geturl(), headers={'Authorization': 'KakaoAK 39938c70588232cbb9c25b118e6772a3'}).json()
    match_first = result['documents'][0]['address']
    
    return float(match_first['y']),float(match_first['x'])

In [ ]:
#교촌치킨 지점별 거리 dataframe 만들기

seoul = pd.read_csv("c:/data/gyochon_seoul.csv")

In [ ]:
lat = seoul.latitude
long = seoul.longitude

In [ ]:
#haversine 함수 사용하기 위한 tuple형으로 저장
lat_long = []
for i in seoul.index:
    lat_long.append((lat[i],long[i]))

In [ ]:
#haversine함수의 출력을 distance에 저장
distance = []
for i in lat_long:
    for j in lat_long:
        distance.append(haversine(i,j))

In [ ]:
#리스트 쪼게는 함수 선언
def list_chunk(lst, n):
    return [lst[i:i+n] for i in range(0, len(lst), n)]

In [ ]:
len(distance) #45796 = 214 * 214

In [ ]:
#45796개의 요소를 214개씩 자르기
distance_chunked = list_chunk(distance,214)

In [ ]:
#store_name을 인덱스와 컬럼으로 하는 DF생성
final = DataFrame(index = seoul.store_name, columns = seoul.store_name)

In [ ]:
#df 채우기
for i in range(len(final)):
    final.iloc[:,i] = distance_chunked[i]   

In [ ]:
#저장, 한글이 깨지지 않게 하기 위한 encoding
final.to_csv("c:/data/seoul_distance_between.csv",mode = 'w',encoding='euc-kr')

In [ ]:
------------------------------------------------------------------------------------------------------------
# 지점간 거리 가까운 순서대로 만들기 

In [ ]:
#df 생성
near_order = DataFrame(index = final.index, columns = range(len(final)))


for i in range(len(near_order)):
    temp = DataFrame(final.iloc[:,i])
    temp = temp.sort_values(by=temp.columns[0])
    temp.iloc[:,0] = round(temp.iloc[:,0],5)
    temp = temp.astype(str)
    temp = temp.reset_index()
    temp = temp['store_name'] + ' ' +'('+ temp.iloc[:,1] + 'Km' + ')'
    near_order.iloc[i,:] = temp

In [ ]:
#본인과의 거리를 가지는 행 제거
del near_order[0]

In [ ]:
near_order.to_csv("c:/data/near_order.csv",mode = 'w',encoding='euc-kr')

In [ ]:
-----------------------------------------------------------------------------------------------------------
#서울 위도 경계값  37.413294
#각 치킨집을 지도에 시각화 하기위한 위도 경도 정보 저장

In [ ]:
#네네치킨 위도 경도
nene = pd.read_csv("c:/data/네네치킨.csv")

nene_latlong = []
for addr in nene['store_address']:
    try:
        nene_latlong.append(getLatLng(addr))
    except:
        pass

len(nene_latlong) #168 중 166

In [ ]:
#훌랄라치킨 위도 경도
hulala = pd.read_csv("c:/data/훌랄라참숯불치킨.csv")

hulala_latlong = []
for addr in hulala['store_address']:
    try:
        hulala_latlong.append(getLatLng(addr))
    except:
        pass

len(hulala_latlong) #30 중 28

In [ ]:
#처가집치킨 위도 경도
cheoga = pd.read_csv("c:/data/처갓집.csv")

cheoga_latlong =[]
for addr in cheoga['store_address']:
    try:
        cheoga_latlong.append(getLatLng(addr))
    except:
        pass

len(cheoga_latlong) #132 중 125

In [ ]:
#페리카나 위도 경도
pelicana = pd.read_csv("c:/data/페리카나.csv")

pelicana_latlong = []
for addr in pelicana['store_address']:
    try:
        pelicana_latlong.append(getLatLng(addr))
    except:
        pass

len(pelicana)
len(pelicana_latlong) #152 중 147

In [ ]:
#멕시카나
mexicana = pd.read_csv("c:/data/멕시카나.csv")

mexicana_latlong = []
for addr in mexicana['store_address']:
    try:
        mexicana_latlong.append(getLatLng(addr))
    except:
        pass

len(mexicana_latlong) #70 중 68

In [ ]:
#굽네 치킨
gubne = pd.read_csv("c:/data/굽네치킨.csv")

gubne_latlong = []
for addr in gubne['store_address']:
    try:
        gubne_latlong.append(getLatLng(addr))
    except:
        pass

len(gubne_latlong) #171 중 155

In [ ]:
#BBQ
bbq = pd.read_csv("c:/data/BBQ.csv")

bbq_latlong = []
for addr in bbq['store_address']:
    try:
        bbq_latlong.append(getLatLng(addr))
    except:
        pass

len(bbq_latlong) #301 중 287

In [ ]:
#구 구분이 담긴 json file
state_geo = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'

map = folium.Map(location=[37.715133,126.734086]) #[위도,경도]

In [ ]:
#서울시 구 구분
folium.Choropleth(
    geo_data = state_geo,
    fill_opacity=0,
    line_color='black',
    name = 'choropleth').add_to(map)

In [ ]:
#교촌치킨 500m 반지름 원 그리기
for i in seoul.index:
    folium.Circle(lat_long[i],
                    radius=500,
                    fill = True,
                    fill_opacity = 0.4,
                    weight = 0,              
                   ).add_to(map)

for i in seoul.index:
    folium.Circle(lat_long[i],
                    radius=1000,
                    fill = True,
                    fill_opacity = 0.2,
                    weight = 0,
                   ).add_to(map)

for i in seoul.index:
    folium.Circle(lat_long[i],
                    radius=1500,
                    fill = True,
                    fill_opacity = 0.1,
                    weight = 0,
                   ).add_to(map)

In [ ]:
#교촌치킨 마커 추가    
for i in seoul.index:
    folium.Marker(lat_long[i],
                  popup=str(seoul['store_name'][i]),
                  icon=folium.Icon(color='blue',icon='star')).add_to(map)

#네네치킨 마커 추가
for i in range(len(nene_latlong)):
    folium.Marker(nene_latlong[i],
                  popup='네네치킨',
                  icon=folium.Icon(color='red',icon='info-sign')).add_to(map)

#훌랄라참숯불치킨 마커 추가
for i in range(len(hulala_latlong)):
    folium.Marker(hulala_latlong[i],
                  popup='훌랄라참숯불치킨',
                  icon=folium.Icon(color='red',icon='info-sign')).add_to(map)

#처갓집 마커 추가
for i in range(len(cheoga_latlong)):
    folium.Marker(cheoga_latlong[i],
                  popup='처갓집',
                  icon=folium.Icon(color='red',icon='info-sign')).add_to(map)

#페리카나 마커 추가
for i in range(len(pelicana_latlong)):
    folium.Marker(pelicana_latlong[i],
                  popup='페리카나',
                  icon=folium.Icon(color='red',icon='info-sign')).add_to(map)


#멕시카나 마커 추가
for i in range(len(mexicana_latlong)):
    folium.Marker(mexicana_latlong[i],
                  popup='멕시카나',
                  icon=folium.Icon(color='red',icon='info-sign')).add_to(map)


#굽네 마커 추가
for i in range(len(gubne_latlong)):
    folium.Marker(gubne_latlong[i],
                  popup='굽네치킨',
                  icon=folium.Icon(color='red',icon='info-sign')).add_to(map)


#BBQ 마커 추가
for i in range(len(bbq_latlong)):
    folium.Marker(bbq_latlong[i],
                  popup='BBQ',
                  icon=folium.Icon(color='red',icon='info-sign')).add_to(map)


map.save("c:/data/test.html")

In [ ]:
----------------------------------------------------------------------------------------------------------------
#상권_코드_명에서 동 추출하기

In [ ]:
seoul = pd.read_csv("c:/data/서울시 우리마을가게 상권분석서비스(상권-추정유동인구).csv",encoding = 'euc-kr')

In [ ]:
seoul['기준 년코드'].value_counts()

In [ ]:
#2020년도 데이터만 사용
seoul = seoul[seoul['기준 년코드'] ==  2020]
code_name = seoul['상권_코드_명']

code_name = '서울' + ' ' + code_name

In [ ]:
#키워드 입력하면 주소에서 동을 뽑아내는 함수
def getDong(keyword):
    searching = keyword
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
    headers = {
        "Authorization": "KakaoAK 39938c70588232cbb9c25b118e6772a3"
    }
    places = requests.get(url, headers = headers).json()['documents'][0]['address_name']
    dong = re.findall('구 .+동', places)
    dong = dong[0].replace('구 ','')
    return dong

In [ ]:
#api사용해 구 따오기 1496:2020년 1분기 상권 수
code_dong = []
cnt = 0

for i in range(1496):
    cnt +=1
    try:
        code_dong.append(getDong(code_name[i]))
    except:
        code_dong.append(code_name[i])
        
    if cnt %10 ==0:
        print(cnt)


x = code_dong

x = DataFrame(x)
x.to_csv("c:/data/2020유동인구 동별.csv",encoding = 'euc-kr')